In [1]:
import numpy as np
from scipy.stats import unitary_group
#fixed!!
nDim = 2
Ns = 2
Nc = 2

Nx = 4
Nt = 4
lattice_volume = Nt * Nx

In [2]:
Pauli = []
Pauli.append(np.array([[0, 1], [1, 0]]))
Pauli.append(np.array([[0,-1j], [1j, 0]]))
Pauli.append(np.array([[1, 0], [0, -1]]))
Id = np.array([[1, 0], [0, 1]])
gamma5 = 1j*np.matmul(Pauli[2], Pauli[0])

In [3]:
def Transpose(array):
    axes = np.arange(len(array.shape))
    axes[-2:] = np.flip(axes[-2:]) 
    return np.transpose(array, axes = axes)

In [4]:
def ConjugateTranspose2(array):
    axes = np.arange(len(array.shape))
    axes[-2:] = np.flip(axes[-2:]) 
    return np.conjugate(np.transpose(array, axes = axes))

In [5]:
def RandomSU2matrix():
    tmp = np.random.rand(3)
    a = np.sqrt(sum(tmp*tmp))
    tmp /= a
    out = np.array([[0.j,0.j],[0.j,0.j]])
    for i in range(3):
        out += Pauli[i] * tmp[i]
    out *= np.sin(a) * 1j
    return Id * np.cos(a) + out

Dirac Lagrangian on the lattice using Wilson fermions:

$\overline{\psi}_x D_{xy}[U] \psi_y = \overline{\psi}_x \left[ (m_q+2) \delta_{xy} -\frac{1}{2}\sum_{\mu} \left(\Gamma_{+\mu} U_\mu(x) \delta_{x+\hat\mu,y} + \Gamma_{-\mu} U^\dagger_\mu(x-\hat\mu) \delta_{x-\hat\mu,y} \right) \right] \psi_y$,

where $\Gamma_{\pm \mu} = \mathbb{1} \mp \gamma_\mu$, $\gamma_\mu$ are the Dirac $\gamma$-matrices, $U_\mu(x)$ is the gauge link at site $x$ pointing at direction $\mu$, and $\hat\mu$ is a unit vector pointing in direction $\mu$.

Note that the $\Gamma$'s are $2 \times 2$ matrices in 2 spacetime dimensions. Dirac and colour indices have been suppressed in the above formula for clarity.

Calculating D, and inverting D is numerically slow, therefore we use the approximation where we can expand the inverse acting on a vector by a sum over applying D multiple times to this same vector. For this we need to Calculate D, which is sparse, and then compute the Matrix-Vector multiplication, which can be done for CSR Sparse matrices. Other way to do this (maybe not possible in the optical hardware?) is to directly apply D to the vector, and this is also implemented

In [6]:
def RandomGaugeLinks(lattice_volume, nDim, Nc):
    gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
    for i in range(lattice_volume):
        for j in range(nDim):
            gauge_links[i,j] = unitary_group.rvs(Nc)
            
    return gauge_links

In [7]:
psi = np.random.randn(lattice_volume, 2, 2)

In [8]:
gauge_links = np.zeros((lattice_volume, nDim, 2,2),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = RandomSU2matrix()

In [9]:
def LinearToLattice(array, Nt, Nx):
    return np.reshape(array, np.roll(np.append(np.array(array[0].shape), [Nx,Nt]),2))
def LatticeToLinear(array, lattice_volume):
    return np.reshape(array, np.roll(np.append(np.array(array[0,0].shape), lattice_volume),1))

# Sparse Matrix Multiplication

In [10]:
def CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli):
    #Delta_x_y, dependent only on the onsite
    diagonal = (m+2)
    
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]
    
    gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
    gauge_links = LatticeToLinear(gauge_links, lattice_volume) * (-0.5)
    gauge_links_shifted_t = LatticeToLinear(gauge_links_shifted_t, lattice_volume) * (-0.5)
    gauge_links_shifted_x = LatticeToLinear(gauge_links_shifted_x, lattice_volume) * (-0.5)
    
    
    #In each row they are 2^d (next Neighbors) + 1 (onsite)
    row_index = np.arange(0,(2**nDim+1)*lattice_volume+1, 2**nDim+1)


    col_index = np.array([])
    values_spinors = np.array([])
    values_colors = np.array([])
    for i in range(lattice_volume):
        #Just moving in time and space to the next neighbors considering periodic boundary
        col_index = np.append(col_index,
                                 np.array([(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt
                                           ,(np.floor(i/4)*4)+((i%Nx)-1)%Nx, i,
                                           (np.floor(i/4)*4)+((i%Nx)+1)%Nx, (i%Nx) +((np.floor((i/Nt))+1)%Nt)*Nt]))
        #Choose the right values depending on direction in space-time, according to the order from above in col_index
        
        values_spinors = np.append(values_spinors, np.array([offdiagonal_spinor_t_minus, 
                                         offdiagonal_spinor_x_minus, np.sqrt(diagonal)*np.identity(Ns), 
                                                             offdiagonal_spinor_x_plus, offdiagonal_spinor_t_plus]))
        #We have to use sqrt of diagonal instead of diagonal, because we perform two matrix multiplication
        
#         values_colors = np.append(values_colors, np.array([gauge_links[(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt,0].T, 
#                                          gauge_links[(np.floor(i/4)*4)+((i%Nx)-1)%Nx,1].T, diagonal*np.identity(Nc), 
#                                                              gauge_links[i,1], 
#                                                            offdiagonal_spinor_t_plus]))
        values_colors = np.append(values_colors, np.array([ConjugateTranspose2(gauge_links_shifted_t[i]), 
                                         ConjugateTranspose2(gauge_links_shifted_x[i]), np.sqrt(diagonal)*np.identity(Nc), 
                                                             gauge_links[i,1], gauge_links[i,0]]))
    
        #I feel the transpose should be opposite, or idk
        
        #Maybe it's smart to add a value_diagonal, and separate all three, we could eliminate the diagonal term in col
        #and then we can choose the correct gauge link using the col_index. Or maybe we could move the gauge_link to right
        #The problem with the first approach, is that the positive u dont shift the gauge link. Therefore, maybe only
        #considere specific elements of col_index, namely the minus part and the diagonal part. 
        
        
    values_spinors = np.reshape(values_spinors, (lattice_volume*(2**nDim+1), Ns,Ns))
    values_colors = np.reshape(values_colors, (lattice_volume*(2**nDim+1), Nc,Nc))
    return row_index.astype(int), col_index.astype(int), values_spinors, values_colors

In [11]:
def CalculateD_flatten(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli):
    #Delta_x_y, dependent only on the onsite
    diagonal = (m+2)
   
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]
   
    gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
    gauge_links = LatticeToLinear(gauge_links, lattice_volume) * (-0.5)
    gauge_links_shifted_t_T = ConjugateTranspose2(LatticeToLinear(gauge_links_shifted_t, lattice_volume) * (-0.5)) #.T correct?
    gauge_links_shifted_x_T = ConjugateTranspose2(LatticeToLinear(gauge_links_shifted_x, lattice_volume) * (-0.5))
   
    offdiag_x_plus = np.kron(gauge_links[:,1,:,:],offdiagonal_spinor_x_plus) #Careful with U dimension t,x
    offdiag_x_minus = np.kron(gauge_links_shifted_x_T,offdiagonal_spinor_x_minus)
    offdiag_t_plus = np.kron(gauge_links[:,0,:,:],offdiagonal_spinor_t_plus)
    offdiag_t_minus = np.kron(gauge_links_shifted_t_T,offdiagonal_spinor_t_minus)

    #In each row they are 2^d (next Neighbors) + 1 (onsite)
    row_index = np.arange(0,(2**nDim+1)*lattice_volume+1, 2**nDim+1)


    col_index = np.array([])
    values = np.array([])

    for i in range(lattice_volume):
        #Just moving in time and space to the next neighbors considering periodic boundary
        col_index = np.append(col_index,
                                 np.array([(i%Nx) +((np.floor((i/Nt))-1)%Nt)*Nt
                                           ,(np.floor(i/4)*4)+((i%Nx)-1)%Nx, i,
                                           (np.floor(i/4)*4)+((i%Nx)+1)%Nx, (i%Nx) +((np.floor((i/Nt))+1)%Nt)*Nt]))
        #Choose the right values depending on direction in space-time, according to the order from above in col_index
       
        values = np.append(values, np.array([offdiag_t_minus[i],
                                         offdiag_x_minus[i], diagonal*np.identity(Ns*Nc),
                                                             offdiag_x_plus[i], offdiag_t_plus[i]]))

       
    values = np.reshape(values, (lattice_volume*(2**nDim+1), Ns*Nc,Ns*Nc))
    return row_index.astype(int), col_index.astype(int), values


In [12]:
def SparseMatrixVectorMultiplication(rows, cols, vals, input_vec):
    out_vec = np.zeros(input_vec.shape,dtype = 'complex_')
    for i in range(input_vec[:,0].size):
        row_elem = rows[i]
        num_elements = rows[i+1] - rows[i]

        output = np.zeros(input_vec[0].shape,dtype = 'complex_')
        for j in range(num_elements):
            output = output + np.matmul(vals[row_elem + j],  input_vec[cols[row_elem + j]])
        out_vec[i] = output
    
    return out_vec

In [13]:
def SparseMatrixVectorMultiplication2(rows, cols, values_spinors, values_colors, input_vec):
    out_vec = np.zeros(input_vec.shape,dtype = 'complex_')
    for i in range(input_vec[:,0,0].size):
        row_elem = rows[i]
        num_elements = rows[i+1] - rows[i]

        output = np.zeros(input_vec[0].shape,dtype = 'complex_')
        for j in range(num_elements):
            output = output + np.matmul(np.matmul(values_colors[row_elem + j],  input_vec[cols[row_elem + j]]),
                                        values_spinors[row_elem + j].T)
        out_vec[i] = output
    
    return out_vec

#### Test random U

In [14]:
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)
rows, cols, values = CalculateD_flatten(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new_flatten = SparseMatrixVectorMultiplication(rows, cols, values, psi_flatten)
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication2(rows, cols, values_spinors, values_colors, psi)

In [15]:
psi_new_reshaped = np.reshape(psi_new_flatten, (lattice_volume,Nc,Ns))
np.sqrt(np.sum((psi_new-psi_new_reshaped)**2))

(6.067498337284559e-15+0j)

#### Test unitary U

In [16]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = unitary_group.rvs(Nc)
        
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
rows, cols, values = CalculateD_flatten(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new_flatten = SparseMatrixVectorMultiplication(rows, cols, values, psi_flatten)
rows, cols, values_spinors, values_colors = CalculateD(Nx,Nt,Ns,Nc,lattice_volume,gauge_links,m,Pauli)
psi_new = SparseMatrixVectorMultiplication2(rows, cols, values_spinors, values_colors, psi)
psi_new_reshaped = np.reshape(psi_new_flatten, (lattice_volume,Nc,Ns))
np.sqrt(np.sum((psi_new-psi_new_reshaped)**2))

(4.203427098877386e-15+1.4331898054452267e-16j)

# Apply D directly

In [17]:
def applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    if len(gauge_links) ==lattice_volume:
        gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    if len(psi) ==lattice_volume:
        psi = LinearToLattice(psi, Nt,Nx)

    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]

    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)

    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    psi_shifted_m_t = np.roll(psi, 1, axis = 0)
    psi_shifted_p_t = np.roll(psi, -1, axis = 0)
    psi_shifted_m_x = np.roll(psi, 1, axis = 1)
    psi_shifted_p_x = np.roll(psi, -1, axis = 1)


    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    time_contribution_m = np.matmul(ConjugateTranspose2(gauge_links_shifted_t), psi_shifted_m_t)
    time_contribution_p = np.matmul(gauge_links[:,:,0,:,:], psi_shifted_p_t)
    space_contribution_m = np.matmul(ConjugateTranspose2(gauge_links_shifted_x), psi_shifted_m_x)
    space_contribution_p = np.matmul(gauge_links[:,:,1,:,:], psi_shifted_p_x)

    # Estos matmuls son diferentes a los matmul
    time_contribution_m = np.matmul(time_contribution_m, Transpose(offdiagonal_spinor_t_minus))
    time_contribution_p = np.matmul(time_contribution_p, Transpose(offdiagonal_spinor_t_plus))
    space_contribution_m = np.matmul(space_contribution_m, Transpose(offdiagonal_spinor_x_minus))
    space_contribution_p = np.matmul(space_contribution_p, Transpose(offdiagonal_spinor_x_plus))
    

    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* (time_contribution_m + 
                                       time_contribution_p + space_contribution_m + space_contribution_p)
    
    return psi_new

In [18]:
def applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    if len(gauge_links) ==lattice_volume:
        gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    if len(psi) ==lattice_volume:
        psi = LinearToLattice(psi, Nt,Nx)
    print(gauge_links.shape)
    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]

    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
   
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]


   

   
   
   
    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    psi_shifted_m_t = np.roll(psi, 1, axis = 0)
    psi_shifted_p_t = np.roll(psi, -1, axis = 0)
    psi_shifted_m_x = np.roll(psi, 1, axis = 1)
    psi_shifted_p_x = np.roll(psi, -1, axis = 1)


    offdiag_x_plus = np.kron(gauge_links[:,:,1,:,:],offdiagonal_spinor_x_plus) #Careful with U dimension t,x
    offdiag_x_minus = np.kron(ConjugateTranspose2(gauge_links_shifted_x),offdiagonal_spinor_x_minus)
    offdiag_t_plus = np.kron(gauge_links[:,:,0,:,:],offdiagonal_spinor_t_plus)
    offdiag_t_minus = np.kron(ConjugateTranspose2(gauge_links_shifted_t),offdiagonal_spinor_t_minus)
   

    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    time_contribution_m = np.matmul(offdiag_t_minus, psi_shifted_m_t[:,:,:,None]) #Do this without None
    time_contribution_p = np.matmul(offdiag_t_plus, psi_shifted_p_t[:,:,:,None])
    space_contribution_m = np.matmul(offdiag_x_minus, psi_shifted_m_x[:,:,:,None])
    space_contribution_p = np.matmul(offdiag_x_plus, psi_shifted_p_x[:,:,:,None])



    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* np.squeeze((time_contribution_m +  #Squeeze is weird!
                                       time_contribution_p + space_contribution_m + space_contribution_p))
   
    return psi_new

#### Test random U

In [19]:
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
gauge_links = np.random.randn(lattice_volume, nDim, Nc,Nc)
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
psi_new2_flatten = applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi_flatten)

psi_new2_reshaped = np.reshape(psi_new2_flatten, (Nt,Nx,Nc,Ns))
np.sqrt(np.sum((psi_new2 - psi_new2_reshaped)**2))

(4, 4, 2, 3, 3)


4.368131539346454e-15

#### Test unitary U

In [20]:
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = unitary_group.rvs(Nc)
Ns = 2
Nc = 3
m = 1
psi = np.random.randn(lattice_volume,Nc,Ns)
psi_flatten = np.reshape(psi, (lattice_volume, Nc*Ns))
psi_new2 = applyD(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi)
psi_new2_flatten = applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi_flatten)

psi_new2_reshaped = np.reshape(psi_new2_flatten, (Nt,Nx,Nc,Ns))
np.sqrt(np.sum((psi_new2 - psi_new2_reshaped)**2))

(4, 4, 2, 3, 3)


(1.235525559952609e-15-7.232804590703526e-17j)

In [21]:
def MatMul(matrix1, matrix2, shape1, shape2):
    first_index = 0
    while len(shape1)>len(shape2):
        shape2.append(1)
    if shape1[0] == shape2[0]:
        dimension_flattened1 = shape1[1]*shape1[2]
        dimension_flattened2 = shape2[1]*shape2[2]
        dimension_flattened_result = shape1[1]*shape2[2]

        matrix_result = np.zeros((shape1[0]* dimension_flattened_result), dtype = 'complex_')

        for lattice_number in range(shape1[0]):
            for first_index in range(shape1[1]):
                for second_index in range(shape2[2]):
                    start_index1 = lattice_number*dimension_flattened1
                    array1 = matrix1[start_index1 + first_index*shape1[2] : start_index1 + (first_index+1)*shape1[2]]


                    #Here, the second_index should be used for a general matrix multiplication, if needed
                    start_index2 = lattice_number*dimension_flattened2
                    array2 = matrix2[start_index2 + second_index*shape2[1] : start_index2 + (second_index+1)*shape2[1]]


                    start_index = (lattice_number*dimension_flattened_result) +first_index*shape2[2]
                    end_index = (lattice_number*dimension_flattened_result) + (first_index+1)*shape2[2]
                    matrix_result[start_index : end_index] = np.dot(array1,array2)


            
    return matrix_result


dim1 = 56
dim2 = 100
matrix1 = np.random.randn(16*dim1*dim2)
matrix2 = np.random.randn(16*dim2)
shape1 = [16,dim1,dim2]
shape2 = [16,dim2]

matrix1_rs = np.reshape(matrix1,shape1)
matrix2_rs = np.reshape(matrix2,shape2)

matrix_result = MatMul(matrix1, matrix2, shape1, shape2)
matrix_result_reshaped = np.reshape(matrix_result, (16,dim1,1))
matrix_result_matmul = np.matmul(matrix1_rs,matrix2_rs[:,:,None])
print(np.sqrt(np.sum((matrix_result_matmul - matrix_result_reshaped)**2)))
print(matrix_result.shape)

(7.021719254729954e-14+0j)
(896,)


In [22]:
def Roll_t(matrix,shape,roll, Nt,Nx):
    if len(shape)==3:
        shape.append(1)
    matrix_result = np.zeros(matrix.shape, dtype = 'complex_')
    rolled_indices = np.roll(np.arange(Nt),roll)
    size0 = shape[1]*shape[2]*shape[3]
    for t in range(Nt):
        start_index = t*size0
        end_index = (t+1)*size0

        start_index_rolled = rolled_indices[t] *size0
        end_index_rolled = (rolled_indices[t]+1) *size0
        matrix_result[start_index : end_index] = matrix[start_index_rolled: end_index_rolled]
    return matrix_result

Nt2 = 2
Nx2 = 10
lattice_volume2 = Nt2*Nx2
ind1 = 5
ind2 = 1

matrix = np.random.randn(lattice_volume2*ind1*ind2)
shape = [Nt2,Nx2,ind1]
matrixrs = np.reshape(matrix, shape)

matrix_result = Roll_t(matrix,shape,-1,Nt2,Nx2)
matrix_result_reshaped = np.reshape(matrix_result, shape)
matrix_rolled = np.reshape(np.roll(matrixrs,-1,axis = 0),shape)
np.sqrt(np.sum((matrix_result_reshaped - matrix_rolled)**2))

0j

In [23]:
def Roll_x(matrix, shape, roll, Nt,Nx):
    if len(shape)==3:
        shape.append(1)
    matrix_result = np.zeros(matrix.shape, dtype = 'complex_')
    
    size_matrix = shape[2]*shape[3]
    size0 = shape[1]*size_matrix
    
    indices = np.arange(Nx)
    rolled_indices = np.roll(indices,roll)
    
    for t in range(Nt):
        for x in range(Nx):
            start_index = t*size0
            end_index = (t+1)*size0
            array_t = matrix[start_index: end_index]
            
            start_index_x = x*size_matrix
            end_index_x = (x+1)*size_matrix
            start_index_rolled = rolled_indices[x] *size_matrix
            end_index_rolled = (rolled_indices[x]+1) *size_matrix
            rolled_array = array_t[start_index_rolled: end_index_rolled]

            matrix_result[start_index+start_index_x : start_index+ end_index_x] = rolled_array
    return matrix_result

Nt2 = 4
Nx2 = 4
ind1 = 6
ind2 = 6

matrix = np.random.randn(Nt2*Nx2*ind1*ind2)
#matrix = np.arange(Nt2*Nx2*ind1*ind2)
shape = [Nt2,Nx2,ind1,ind2]

matrixrs = np.reshape(matrix, shape)

matrix_rolled = np.roll(matrixrs,1,axis = 1)
matrix_result = Roll_x(matrix,shape,1,Nt2,Nx2)
matrix_result_reshaped = np.reshape(matrix_result, shape)
np.sqrt(np.sum((matrix_result_reshaped - matrix_rolled)**2))

0j

In [24]:
def Roll_x2(matrix,shape,roll, Nt,Nx):
    if len(shape)==3:
        shape.append(1)
    matrix_result = np.zeros(matrix.shape, dtype = 'complex_')
    
    size_matrix = shape[2]*shape[3]
    size0 = shape[1]*size_matrix
    
    indices_t = np.arange(Nt) *size0
    
    indices_x = np.arange(Nx)*size_matrix
    rolled_indices_x = np.roll(indices_x,roll)
    
    indices = np.arange(size_matrix)
     
    rolled_combined_indices = ((indices_t[:,None] + rolled_indices_x[None,:])[:,:,None] + indices[None,:]).flatten()
    #This idea works for a scalar value for each 


    return matrix[rolled_combined_indices]

Nt2 = 16
Nx2 = 20
ind1 = 18
ind2 = 3

matrix = np.random.randn(Nt2*Nx2*ind1*ind2)
#matrix = np.arange(Nt2*Nx2*ind1*ind2)
shape = [Nt2,Nx2,ind1,ind2]

matrixrs = np.reshape(matrix, shape)

matrix_rolled = np.roll(matrixrs,-1,axis = 1)
matrix_result = Roll_x2(matrix,shape,-1,Nt2,Nx2)
matrix_result_reshaped = np.reshape(matrix_result, shape)
np.sqrt(np.sum((matrix_result_reshaped - matrix_rolled)**2))

0.0

In [25]:
def Kron(matrix1,matrix2,shape1,shape2):
    size1 = len(matrix1)
    size2 = len(matrix2)

    size_new = shape1[2]*shape1[3]*shape2[0]*shape2[1]


    index_size1 = shape1[2]*shape2[0]
    index_size2 = shape1[3]*shape2[1]

    array_result = np.zeros(size1*size2, dtype = 'complex_')
    for lattice_number in range(shape1[0]*shape1[1]):
        for first_index in range(shape1[2]):
            for dimension in range(shape2[0]):
                for dimension2 in range(shape1[2]):
                    start_index_lattice = lattice_number * size_new

                    start_index_dim =  dimension * index_size1 + dimension2*index_size2*2

                    start_index1 = first_index * shape2[1]
                    end_index1 = (first_index+1) * shape2[1]

                    start_index = start_index_lattice + start_index_dim + start_index1
                    end_index = start_index_lattice + start_index_dim + end_index1

                    start_index_mat1_lattice = lattice_number*shape1[2]*shape1[3] +dimension2*shape1[3]
                    start_index_mat1 = start_index_mat1_lattice + first_index 
                    #end_index_mat1 = start_index_mat1_lattice+first_index+1

                    start_index_mat2 = dimension*shape2[1]
                    end_index_mat2 = (dimension+1)*shape2[1]
                    array_result[start_index: end_index] = matrix1[start_index_mat1] *matrix2[start_index_mat2:end_index_mat2]
    #                 print('start index = ' + str(start_index))
    #                 print('end index = ' + str(end_index))
    #                 print('start index mat 1 = ' + str(start_index_mat1))
    #                 #print(end_index_mat1)
    #                 print('start index mat 2= ' + str(start_index_mat2))
    #                 print('end index mat 2= ' + str(end_index_mat2))
    #                 print('\n')
    return array_result

lattice_volume2 = 20


shape1 = [lattice_volume2,1,3,3] #Corregir esta webada, el 1 ahi esta redundante!
shape2 = [2,2]

offdiagonal_spinor_x_plus = Id - Pauli[1]
matrix2 = offdiagonal_spinor_x_plus.flatten()

gauge_links = np.random.randn(lattice_volume2, nDim, Nc,Nc)
matrix1 = gauge_links[:,1,:,:].flatten()

array_reshaped = np.reshape(Kron(matrix1,matrix2,shape1,shape2), (lattice_volume2,6,6))
offdiag_x_plus = np.kron(gauge_links[:,1,:,:],offdiagonal_spinor_x_plus)
np.sqrt(np.sum((array_reshaped-offdiag_x_plus)**2))

0j

In [26]:
def Transposed_indices(nDim):
    indices = np.arange(nDim)
    repeated_indices = indices*nDim
    return (indices[:,None] + repeated_indices[None,:]).flatten()


def ConjugateTranspose(matrix,lattice_volume,nDim, tranposed_indices): 
    array_result = np.zeros(matrix.shape, dtype = 'complex_')
    size_matrix = nDim**2

    #transposed_indices = np.array([0,3,6,1,4,7,2,5,8]) for 3x3 Matrices

    for lattice_number in range(lattice_volume):
        start_index = lattice_number *size_matrix
        end_index = (lattice_number+1) *size_matrix
        indices = start_index + transposed_indices
        array_result[start_index:end_index] = np.conjugate(matrix[indices])
    return array_result

gauge_links = np.random.randn(lattice_volume2, Nc,Nc) + 1j*np.random.randn(lattice_volume2, Nc,Nc)
matrix = gauge_links.flatten()

transposed_indices = Transposed_indices(Nc)
array_result = ConjugateTranspose(matrix,lattice_volume2,Nc, transposed_indices)

array_reshaped = np.reshape(array_result,(lattice_volume2, 3,3))
transposed = np.conjugate(np.squeeze(Transpose(gauge_links)))
np.sqrt(np.sum((array_reshaped-transposed)**2))

0j

In [27]:
def applyD_flatten_flatten(lattice_volume,Nt, Nx ,gauge_links_t, gauge_links_x,m,Pauli,psi, transposed_indices):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    
    #Correct this
    offdiagonal_spinor_x_plus = (Id - Pauli[0]).flatten()
    offdiagonal_spinor_x_minus = (Id + Pauli[0]).flatten()
    offdiagonal_spinor_t_plus = (Id - Pauli[2]).flatten()
    offdiagonal_spinor_t_minus = (Id + Pauli[2]).flatten()
    
    
    

    shape_gauge_links = [Nt,Nx, Nc,Nc] #Correct this, lattice volume instead of Nt,Nx or just be consistent overall!
    gauge_links_shifted_t = Roll_t(gauge_links_t,shape_gauge_links,1,Nt,Nx)
    gauge_links_shifted_x = Roll_x(gauge_links_x,shape_gauge_links,1,Nt,Nx)
   


   

   
   
   
    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    shape_psi = [Nt,Nx, Nc*Ns]
    psi_shifted_m_t = Roll_t(psi, shape_psi, 1,Nt,Nx)
    psi_shifted_p_t = Roll_t(psi, shape_psi, -1,Nt,Nx)
    psi_shifted_m_x = Roll_x(psi, shape_psi, 1,Nt,Nx)
    psi_shifted_p_x = Roll_x(psi, shape_psi, -1,Nt,Nx)

    shape_gauge = [lattice_volume,1,Nc,Nc]
    shape_gamma = [2,2]
    offdiag_x_plus = Kron(gauge_links_x,offdiagonal_spinor_x_plus, shape_gauge, shape_gamma) #Careful with U dimension t,x
    offdiag_x_minus = Kron(ConjugateTranspose(gauge_links_shifted_x,lattice_volume, Nc, transposed_indices),offdiagonal_spinor_x_minus, shape_gauge, shape_gamma)
    offdiag_t_plus = Kron(gauge_links_t, offdiagonal_spinor_t_plus, shape_gauge, shape_gamma)
    offdiag_t_minus = Kron(ConjugateTranspose(gauge_links_shifted_t,lattice_volume, Nc, transposed_indices),offdiagonal_spinor_t_minus, shape_gauge, shape_gamma)

    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    shape_psi = [lattice_volume, Nc*Ns] #really be consistent with this shit
    shape_kron = [lattice_volume, Nc*Ns,Nc*Ns]
    time_contribution_m = MatMul(offdiag_t_minus, psi_shifted_m_t, shape_kron, shape_psi) #Do this without None
    time_contribution_p = MatMul(offdiag_t_plus, psi_shifted_p_t, shape_kron, shape_psi)
    space_contribution_m = MatMul(offdiag_x_minus, psi_shifted_m_x, shape_kron, shape_psi)
    space_contribution_p = MatMul(offdiag_x_plus, psi_shifted_p_x, shape_kron, shape_psi)



    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* (time_contribution_m +  #Squeeze is weird!
                                       time_contribution_p + space_contribution_m + space_contribution_p)
   
    return psi_new



Nt = 4
Nx = 4
lattice_volume = Nt*Nx
gauge_links = np.random.randn(lattice_volume,2, Nc,Nc)
gauge_links_t = gauge_links[:,0,:,:].flatten()
gauge_links_x = gauge_links[:,1,:,:].flatten()
m = 1
psi = np.random.randn(lattice_volume,Nc*Ns)
psi_flatten = psi.flatten()




result_flattened_operation_reshaped = np.reshape(applyD_flatten_flatten(lattice_volume,Nt, Nx ,gauge_links_t, gauge_links_x,m,Pauli,psi_flatten,transposed_indices),(lattice_volume,Nc*Ns))

result = np.reshape(applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi), (lattice_volume, Nc*Ns))

np.sqrt(np.sum((result-result_flattened_operation_reshaped)**2))

(4, 4, 2, 3, 3)


(3.5388086795394242e-15+0j)

In [28]:
def Roll_x2(matrix,shape,roll, Nt,Nx):
    if len(shape)==3:
        shape.append(1)
    matrix_result = np.zeros(matrix.shape, dtype = 'complex_')
    
    size_matrix = shape[2]*shape[3]
    size0 = shape[1]*size_matrix
    
    indices_t = np.arange(Nt) *size0
    
    indices_x = np.arange(Nx)*size_matrix
    rolled_indices_x = np.roll(indices_x,roll)
    
    indices = np.arange(size_matrix)
     
    rolled_combined_indices = ((indices_t[:,None] + rolled_indices_x[None,:])[:,:,None] + indices[None,:]).flatten()
    #This idea works for a scalar value for each 


    return matrix[rolled_combined_indices]

Nt2 = 16
Nx2 = 20
ind1 = 18
ind2 = 3

matrix = np.random.randn(Nt2*Nx2*ind1*ind2)
#matrix = np.arange(Nt2*Nx2*ind1*ind2)
shape = [Nt2,Nx2,ind1,ind2]

matrixrs = np.reshape(matrix, shape)

matrix_rolled = np.roll(matrixrs,-1,axis = 1)
matrix_result = Roll_x2(matrix,shape,-1,Nt2,Nx2)
matrix_result_reshaped = np.reshape(matrix_result, shape)
np.sqrt(np.sum((matrix_result_reshaped - matrix_rolled)**2))

0.0

# Pre Indices

In [29]:
def roll_x_indices(Nt,Nx,size_matrix,roll):
    size0 = Nx*size_matrix

    indices_t = np.arange(Nt) *size0

    indices_x = np.arange(Nx)*size_matrix
    rolled_indices_x = np.roll(indices_x,roll)

    indices = np.arange(size_matrix)

    return ((indices_t[:,None] + rolled_indices_x[None,:])[:,:,None] + indices[None,:]).flatten()

def Roll(matrix, indices):
    return matrix[indices]

Nt2 = 20
Nx2 = 40
ind1 = 3
ind2 = 4
roll = 1

matrix = np.random.randn(Nt2*Nx2*ind1*ind2)
#matrix = np.arange(Nt2*Nx2*ind1*ind2)
shape = [Nt2,Nx2,ind1,ind2]

matrixrs = np.reshape(matrix, shape)
up_x = roll_x_indices(Nt2,Nx2,ind1*ind2,roll)
matrix_rolled = np.roll(matrixrs,roll,axis = 1)
matrix_result = Roll(matrix,up_x)
matrix_result_reshaped = np.reshape(matrix_result, shape)
np.sqrt(np.sum((matrix_result_reshaped - matrix_rolled)**2))

0.0

In [30]:
def roll_t_indices(Nt,Nx,size_matrix,roll):
    size0 = Nx*size_matrix

    indices_t = np.arange(Nt) *size0
    rolled_indices_t = np.roll(indices_t,roll)
    
    indices_x = np.arange(Nx)*size_matrix
    

    indices = np.arange(size_matrix)

    return ((rolled_indices_t[:,None] + indices_x[None,:])[:,:,None] + indices[None,:]).flatten()

Nt2 = 16
Nx2 = 20
ind1 = 9
ind2 = 5
roll = 1

matrix = np.random.randn(Nt2*Nx2*ind1*ind2)
#matrix = np.arange(Nt2*Nx2*ind1*ind2)
shape = [Nt2,Nx2,ind1,ind2]

matrixrs = np.reshape(matrix, shape)
up_t = roll_t_indices(Nt2,Nx2,ind1*ind2,roll)
matrix_rolled = np.roll(matrixrs,roll,axis = 0)
matrix_result = Roll(matrix,up_t)
matrix_result_reshaped = np.reshape(matrix_result, shape)
np.sqrt(np.sum((matrix_result_reshaped - matrix_rolled)**2))

0.0

In [31]:
def Transposed_indices(Nt,Nx,nDim):
    indices = np.arange(nDim)
    repeated_indices = indices*nDim
    size_matrix = nDim**2

    
    indices_lattice = np.arange(Nt*Nx)*size_matrix
    #indices_lattice = np.arange(Nt*Nx) * size0
    #print(indices_lattice)
    #print((indices_t[:,None] + indices_x[None,:]))
    return ((indices_lattice[:,None] + indices[None,:])[:,:,None] + repeated_indices[None,:]).flatten()

Nt2 = 4
Nx2 = 4
lattice_volume2 = Nt2*Nx2


def ConjugateTranspose(matrix,transposed_indices): 
    return np.conjugate(matrix[transposed_indices])

gauge_links = np.random.randn(lattice_volume2, Nc,Nc) + 1j*np.random.randn(lattice_volume2, Nc,Nc)
matrix = gauge_links.flatten()

transposed_indices = Transposed_indices(Nt2,Nx2,Nc)
array_result = ConjugateTranspose(matrix,transposed_indices)

array_reshaped = np.reshape(array_result,(lattice_volume2, 3,3))
transposed = np.conjugate(np.squeeze(Transpose(gauge_links)))
np.sqrt(np.sum((array_reshaped-transposed)**2))

0j

In [32]:
def MatMul_indices(Nt,Nx,nDim):
    indices = np.arange(nDim)
    #repeated_indices = indices*nDim
    columns = indices*nDim
    size_matrix = nDim**2

    #print(repeated_indices)
    #columns = indices[:,None] + repeated_indices[None,:]
    indices_lattice = np.arange(Nt*Nx)*size_matrix
    #indices_lattice = np.arange(Nt*Nx) * size0
    #print(indices_lattice)
    #print((indices_t[:,None] + indices_x[None,:]))
#     matmul_indices = np.zeros((nDim, Nt*Nx*nDim), dtype = 'int')
    indices_column = (indices_lattice[:,None] +columns[None, :]).flatten()
    size_columns = Nt*Nx*nDim
#     for numColumn in range(nDim):
#         indices_column = ((indices_t[:,None] + indices_x[None,:])[:,:,None] +columns[None,numColumn, :]).flatten()
#         matmul_indices[numColumn] = indices_column
        
    #Indices for matrix2
    columns2 = np.arange(Nt*Nx)*nDim
    columns_matrix2 = np.repeat(columns2, nDim)
    return indices_column, columns_matrix2


def MatMul(matrix1, matrix2, matmul_indices, columns_matrix2, nDim):
    result = np.zeros(len(matmul_indices), dtype = 'complex_')
    for column in range(nDim):
        result = result + (matrix1[matmul_indices +column]*matrix2[columns_matrix2 + column])           
    return result

dim1 = 10
dim2 = dim1
Nt2 = 10
Nx2 = 10
matrix1 = np.random.randn(Nt2*Nx2*dim1*dim2)
matrix2 = np.random.randn(Nt2*Nx2*dim2)

shape1 = [Nt2*Nx2,dim1,dim2]
shape2 = [Nt2*Nx2,dim2]
matmul_indices, columns_matrix2 = MatMul_indices(Nt2,Nx2,dim2)
print(matmul_indices.shape)
print(columns_matrix2.shape)
matrix1_rs = np.reshape(matrix1,shape1)
matrix2_rs = np.reshape(matrix2,shape2)

matrix_result = MatMul(matrix1, matrix2, matmul_indices,columns_matrix2, dim2)
matrix_result_reshaped = np.reshape(matrix_result, (Nt2*Nx2,dim1,1))
matrix_result_matmul = np.matmul(matrix1_rs,matrix2_rs[:,:,None])
print(np.sqrt(np.sum((matrix_result_matmul - matrix_result_reshaped)**2)))


(1000,)
(1000,)
(1.4460488414604305e-14+0j)


In [33]:
# def MatMul(matrix1, matrix2, matmul_indices):
#     result = np.zeros(len(matmul_indices[0]), dtype = 'complex_')
#     for column in range(len(matmul_indices)):
#         result = result + (matrix1[matmul_indices[column]]*matrix2[column])           
#     return matrix_result

In [34]:
a = np.array([[1,2,3],[4,5,6]])

In [35]:
np.repeat(a,3,1)

array([[1, 1, 1, 2, 2, 2, 3, 3, 3],
       [4, 4, 4, 5, 5, 5, 6, 6, 6]])

In [36]:
gauge_links.shape

(16, 3, 3)

In [37]:
gauge_links = np.reshape(np.arange(16*3*3),(16,3,3))

In [38]:
b = np.repeat(np.repeat(gauge_links,2,2),2,1)
b.flatten().shape

(576,)

In [39]:
k = np.reshape(np.arange(4), (2,2))
d = np.repeat(np.repeat(k,3,0).reshape(1,2*2*3),16*3,0).flatten()

In [40]:
d.shape

(576,)

In [41]:
def Kron_indices(Nt,Nx, Nc,Ns):
    lattice_volume = Nt*Nx
    gauge_links_indices = np.reshape(np.arange(lattice_volume*Nc*Nc),(lattice_volume,Nc,Nc))
    repeated_gauge_links_indices =  np.repeat(np.repeat(gauge_links_indices,Ns,2),Ns,1)
    
    gamma_indices = np.reshape(np.arange(Ns*Ns), (Ns,Ns))
    repeated_gamma_indices = np.repeat(np.repeat(gamma_indices,Nc,0).reshape(1,Ns*Ns*Nc),lattice_volume*Nc,0)
    return repeated_gauge_links_indices.flatten() , repeated_gamma_indices.flatten()

def Kron(matrix1,matrix2,indices1,indices2):
    return matrix1[indices1]*matrix2[indices2]


Nt2 = 60
Nx2 = 50
lattice_volume2 = Nt2*Nx2
Nc2 = 3
Ns2 = 2

gauge_links_indices, gamma_indices = Kron_indices(Nt2,Nx2, Nc2,Ns2)


In [42]:
shape1 = [lattice_volume2,1,Nc2,Nc2] #Corregir esta webada, el 1 ahi esta redundante!
shape2 = [Ns2, Ns2]

offdiagonal_spinor_x_plus = np.random.randn(Ns2,Ns2)
matrix2 = offdiagonal_spinor_x_plus.flatten()

gauge_links = np.random.randn(lattice_volume2, nDim, Nc2,Nc2)
matrix1 = gauge_links[:,1,:,:].flatten()

array_reshaped = np.reshape(Kron(matrix1,matrix2,gauge_links_indices, gamma_indices), (lattice_volume2,Nc2*Ns2,Nc2*Ns2))


In [43]:
offdiag_x_plus = np.kron(gauge_links[:,1,:,:],offdiagonal_spinor_x_plus)
np.sqrt(np.sum((array_reshaped-offdiag_x_plus)**2))

0.0

In [44]:
def applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    if len(gauge_links) ==lattice_volume:
        gauge_links = LinearToLattice(gauge_links, Nt,Nx)
    if len(psi) ==lattice_volume:
        psi = LinearToLattice(psi, Nt,Nx)
    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]

    gauge_links_shifted_t = np.roll(gauge_links[:,:,0], 1, axis = 0)
    gauge_links_shifted_x = np.roll(gauge_links[:,:,1], 1, axis = 1)
   
    offdiagonal_spinor_x_plus = Id - Pauli[0]
    offdiagonal_spinor_x_minus = Id + Pauli[0]
    offdiagonal_spinor_t_plus = Id - Pauli[2]
    offdiagonal_spinor_t_minus = Id + Pauli[2]


   

   
   
   
    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    psi_shifted_m_t = np.roll(psi, 1, axis = 0)
    psi_shifted_p_t = np.roll(psi, -1, axis = 0)
    psi_shifted_m_x = np.roll(psi, 1, axis = 1)
    psi_shifted_p_x = np.roll(psi, -1, axis = 1)


    offdiag_x_plus = np.kron(gauge_links[:,:,1,:,:],offdiagonal_spinor_x_plus) #Careful with U dimension t,x
    offdiag_x_minus = np.kron(ConjugateTranspose2(gauge_links_shifted_x),offdiagonal_spinor_x_minus)
    offdiag_t_plus = np.kron(gauge_links[:,:,0,:,:],offdiagonal_spinor_t_plus)
    offdiag_t_minus = np.kron(ConjugateTranspose2(gauge_links_shifted_t),offdiagonal_spinor_t_minus)
   

    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)
    time_contribution_m = np.matmul(offdiag_t_minus, psi_shifted_m_t[:,:,:,None]) #Do this without None
    time_contribution_p = np.matmul(offdiag_t_plus, psi_shifted_p_t[:,:,:,None])
    space_contribution_m = np.matmul(offdiag_x_minus, psi_shifted_m_x[:,:,:,None])
    space_contribution_p = np.matmul(offdiag_x_plus, psi_shifted_p_x[:,:,:,None])



    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* np.squeeze((time_contribution_m +  #Squeeze is weird!
                                       time_contribution_p + space_contribution_m + space_contribution_p))
   
    return psi_new

In [45]:
def applyD_flatten_flatten(Nt, Nx ,gauge_links_t, gauge_links_x,m,Pauli,psi,
                           down_t_gauge, down_x_gauge, up_t_psi, down_t_psi,
                           down_x_psi, transposed_indices, matmul_indices, columns_matrix2,
                          gauge_links_indices, gamma_indices):
    #I find it easier to broadcast when working with a 2D system, probably not ideal for big calculations, can be corrected
    diagonal = (m+2)
    #Symetric derivative, each dimension multiplied by a gamma matrix, in 2D, and in this choice, they are pauli matrices
    
    #Correct this
    offdiagonal_spinor_x_plus = (Id - Pauli[0]).flatten()
    offdiagonal_spinor_x_minus = (Id + Pauli[0]).flatten()
    offdiagonal_spinor_t_plus = (Id - Pauli[2]).flatten()
    offdiagonal_spinor_t_minus = (Id + Pauli[2]).flatten()
    
    
    

    shape_gauge_links = [Nt,Nx, Nc,Nc] #Correct this, lattice volume instead of Nt,Nx or just be consistent overall!
    gauge_links_shifted_t = Roll(gauge_links_t,down_t_gauge)
    gauge_links_shifted_x = Roll(gauge_links_x,down_x_gauge)
   


   

   
   
   
    #define jx, and jt, for choosing the right values for the neighbours considering periodic boundary
    shape_psi = [Nt,Nx, Nc*Ns]
    psi_shifted_m_t = Roll(psi, down_t_psi)
    psi_shifted_p_t = Roll(psi,up_t_psi)
    psi_shifted_m_x = Roll(psi, down_x_psi)
    psi_shifted_p_x = Roll(psi, up_x_psi)


    offdiag_x_plus = Kron(gauge_links_x,offdiagonal_spinor_x_plus, gauge_links_indices, gamma_indices) #Careful with U dimension t,x
    offdiag_x_minus = Kron(ConjugateTranspose(gauge_links_shifted_x,transposed_indices),offdiagonal_spinor_x_minus, gauge_links_indices, gamma_indices)
    offdiag_t_plus = Kron(gauge_links_t, offdiagonal_spinor_t_plus, gauge_links_indices, gamma_indices)
    offdiag_t_minus = Kron(ConjugateTranspose(gauge_links_shifted_t,transposed_indices),offdiagonal_spinor_t_minus, gauge_links_indices, gamma_indices)

    #Apply the gamma matrices to the sum of NN, (maybe this part is not right in sense of the spin indices, idk)

    time_contribution_m = MatMul(offdiag_t_minus, psi_shifted_m_t, matmul_indices, columns_matrix2, Nc*Ns) #Do this without None
    time_contribution_p = MatMul(offdiag_t_plus, psi_shifted_p_t, matmul_indices, columns_matrix2, Nc*Ns)
    space_contribution_m = MatMul(offdiag_x_minus, psi_shifted_m_x, matmul_indices, columns_matrix2, Nc*Ns)
    space_contribution_p = MatMul(offdiag_x_plus, psi_shifted_p_x, matmul_indices, columns_matrix2, Nc*Ns)



    #Apply psi on the diagonal
    self_contribution = diagonal*psi
    #Add all
    psi_new = self_contribution -0.5* (time_contribution_m +  #Squeeze is weird!
                                       time_contribution_p + space_contribution_m + space_contribution_p)
   
    return psi_new

Nt = 16
Nx = 16
Nc = 10
down_t_gauge = roll_t_indices(Nt,Nx,Nc*Nc,1)
down_x_gauge = roll_x_indices(Nt,Nx,Nc*Nc,1)
            
up_t_psi     = roll_t_indices(Nt,Nx,Nc*Ns,-1)
up_x_psi     = roll_x_indices(Nt,Nx,Nc*Ns,-1)
down_t_psi   = roll_t_indices(Nt,Nx,Nc*Ns,1)
down_x_psi   = roll_x_indices(Nt,Nx,Nc*Ns,1)

transposed_indices = Transposed_indices(Nt,Nx,Nc)

matmul_indices, columns_matrix2 = MatMul_indices(Nt,Nx,Nc*Ns)

gauge_links_indices, gamma_indices = Kron_indices(Nt,Nx, Nc,Ns)

lattice_volume = Nt*Nx

for i in range(20):
    gauge_links =  RandomGaugeLinks(lattice_volume, 2, Nc)
    gauge_links_t = gauge_links[:,0,:,:].flatten()
    gauge_links_x = gauge_links[:,1,:,:].flatten()
    m = 1
    psi = np.random.randn(Nt*Nx,Nc*Ns)
    psi_flatten = psi.flatten()
    result_flattened_operation_reshaped = np.reshape(applyD_flatten_flatten(Nt, Nx ,gauge_links_t, gauge_links_x,m,Pauli,psi_flatten,
                               down_t_gauge, down_x_gauge, up_t_psi, down_t_psi,
                               down_x_psi, transposed_indices, matmul_indices, columns_matrix2,
                              gauge_links_indices, gamma_indices),(Nt*Nx,Nc*Ns))

    result = np.reshape(applyD_flatten(lattice_volume,Nt, Nx ,gauge_links,m,Pauli,psi), (Nt*Nx, Nc*Ns))

    print(np.sqrt(np.sum((result-result_flattened_operation_reshaped)**2)))

(1.1276043365949814e-14+2.295530232770094e-16j)
(1.2538718353331743e-14+4.6693983116555715e-18j)
(1.2241695889024469e-14+4.026272142648848e-16j)
(1.2262838766379756e-14+5.69242833280139e-16j)
(1.2028793093147713e-14-2.865648604786377e-16j)
(1.0939757479614184e-14-4.5719844108921195e-17j)
(1.2209286441497774e-14-9.366781641637424e-17j)
(1.2795178083048946e-14+1.9290640589025083e-16j)
(1.2131976591443213e-14-2.6580805481906803e-16j)
(1.2573093948922025e-14-2.1653338314705063e-16j)
(1.0819451171759677e-14-6.042600207191109e-16j)
(1.1693466362708913e-14-6.264581356160218e-16j)
(1.2887623663118081e-14+6.210737571255858e-17j)
(1.2770000882376714e-14-9.993117407784194e-17j)
(1.1429452842260224e-14+1.9664635535216394e-16j)
(1.2156924808261554e-14+4.107894409614464e-16j)
(1.2885483543720519e-14+1.3170863640744737e-16j)
(1.200271044351781e-14+1.5962353817434063e-16j)
(1.117728307076864e-14-4.55994625219408e-16j)
(1.1512990476841872e-14-8.390925555335243e-17j)


In [46]:
(result-result_flattened_operation_reshaped)**2

array([[ 4.93038066e-32+0.00000000e+00j, -1.23259516e-32-0.00000000e+00j,
        -4.93038066e-32-0.00000000e+00j, ...,
         0.00000000e+00+0.00000000e+00j, -1.23259516e-32-0.00000000e+00j,
        -9.24446373e-33+1.23259516e-32j],
       [-1.23259516e-32+0.00000000e+00j, -1.23259516e-32-0.00000000e+00j,
        -3.08148791e-33-0.00000000e+00j, ...,
        -4.93038066e-32+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         1.23259516e-32-0.00000000e+00j],
       [ 9.24446373e-33+1.23259516e-32j, -4.93038066e-32-0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, ...,
         0.00000000e+00+0.00000000e+00j, -1.97215226e-31-0.00000000e+00j,
        -4.93038066e-32-0.00000000e+00j],
       ...,
       [ 1.84889275e-31+9.86076132e-32j,  0.00000000e+00+0.00000000e+00j,
        -1.97215226e-31+0.00000000e+00j, ...,
         4.62223187e-32+2.46519033e-32j, -9.43705673e-33-0.00000000e+00j,
        -1.23259516e-32-0.00000000e+00j],
       [-1.23259516e-32-0.00000000e+00j,  0.

In [47]:
def RandomGaugeLinks(lattice_volume, nDim, Nc):
    gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
    for i in range(lattice_volume):
        for j in range(nDim):
            gauge_links[i,j] = unitary_group.rvs(Nc)
            
    return gauge_links

In [48]:
print(down_t_psi)

[4800 4801 4802 ... 4797 4798 4799]


In [49]:
psi.shape

(256, 20)

In [50]:
gauge_links.shape

(256, 2, 10, 10)

# Test

In [51]:
Nt = 4
Nx = 4
Nc = 3
down_t_gauge = roll_t_indices(Nt,Nx,Nc*Nc,1)
down_x_gauge = roll_x_indices(Nt,Nx,Nc*Nc,1)
            
up_t_psi     = roll_t_indices(Nt,Nx,Nc*Ns,-1)
up_x_psi     = roll_x_indices(Nt,Nx,Nc*Ns,-1)
down_t_psi   = roll_t_indices(Nt,Nx,Nc*Ns,1)
down_x_psi   = roll_x_indices(Nt,Nx,Nc*Ns,1)

transposed_indices = Transposed_indices(Nt,Nx,Nc)

matmul_indices, columns_matrix2 = MatMul_indices(Nt,Nx,Nc*Ns)

gauge_links_indices, gamma_indices = Kron_indices(Nt,Nx, Nc,Ns)

lattice_volume = Nt*Nx

U = unitary_group.rvs(3)
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = U
        
gamma_t_plus = np.identity(Ns) - Pauli[2] 
gamma_t_minus = np.identity(Ns) + Pauli[2] 
gamma_x_plus = np.identity(Ns) - Pauli[0] 
gamma_x_minus = np.identity(Ns) + Pauli[0]

gauge_links_t = gauge_links[:,0,:,:].flatten()
gauge_links_x = gauge_links[:,1,:,:].flatten()

for nt in range(5):
    for nx in range(5):
        a = 1
        p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
        #U = np.array([1,0,0])
        psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
        vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
        #vorfaktor[0,0] = 1
        #vorfaktor[0,1] = 1
        for t in range(Nt):
            for x in range(Nx):
                psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
        psi_flatten = psi.flatten()
        prefactor_diagonal = (m + 2)*np.identity(Ns)  
        prefactor_plus = (-0.5*gamma_x_plus*np.round(np.exp(1j*a*p[1]))) -(0.5*gamma_t_plus*np.round(np.exp(1j*a*p[0])))
        prefactor_minus = (-0.5*gamma_x_minus*np.round(np.exp(-1j*a*p[1]))) - (0.5*gamma_t_minus*np.round(np.exp(-1j*a*p[0])))
        manual_calculation_diagonal = Transpose(np.matmul(prefactor_diagonal, Transpose(psi)))
        manual_calculation_plus = np.matmul(U,Transpose(np.matmul(prefactor_plus, Transpose(psi))))
        manual_calculation_minus = np.matmul(ConjugateTranspose2(U),Transpose(np.matmul(prefactor_minus, Transpose(psi))))
        manual_calculation = manual_calculation_diagonal + manual_calculation_plus + manual_calculation_minus
        
        psi_new2 = applyD_flatten_flatten(Nt, Nx ,gauge_links_t, gauge_links_x,m,Pauli,psi_flatten,
                               down_t_gauge, down_x_gauge, up_t_psi, down_t_psi,
                               down_x_psi, transposed_indices, matmul_indices, columns_matrix2,
                              gauge_links_indices, gamma_indices)
        
        
        psi_new2 = np.reshape(psi_new2, (Nt,Nx,Nc,Ns))
        print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))

4.965068306494546e-16j
0j
(1.7488282991824665e-15-2.2553983875655045e-16j)
0j
4.965068306494546e-16j
0j
0j
0j
0j
0j
(1.7488282991824665e-15-2.2553983875655045e-16j)
0j
4.965068306494546e-16j
0j
(1.7488282991824665e-15-2.2553983875655045e-16j)
0j
0j
0j
0j
0j
4.965068306494546e-16j
0j
(1.7488282991824665e-15-2.2553983875655045e-16j)
0j
4.965068306494546e-16j


In [52]:
nt = 0
nx = 0


a = 1
p = np.array([2*np.pi*nt/Nt,2*np.pi*nx/Nx])
#U = np.array([1,0,0])
psi =np.zeros((Nt,Nx, Nc,Ns),dtype = 'complex_')
vorfaktor = np.ones((Nc,Ns), dtype = 'complex_') #np.zeros((Nc,Ns), dtype = 'complex_')
#vorfaktor[0,0] = 1
#vorfaktor[0,1] = 1
for t in range(Nt):
    for x in range(Nx):
        psi[t,x] = np.round(np.exp(1j* np.dot(p,a*np.array([t,x]))) * vorfaktor)#np.ones((Nc,Ns), dtype = 'complex_')
psi_flatten = psi.flatten()
prefactor_diagonal = (m + 2)*np.identity(Ns)  
prefactor_plus = (-0.5*gamma_x_plus*np.round(np.exp(1j*a*p[1]))) -(0.5*gamma_t_plus*np.round(np.exp(1j*a*p[0])))
prefactor_minus = (-0.5*gamma_x_minus*np.round(np.exp(-1j*a*p[1]))) - (0.5*gamma_t_minus*np.round(np.exp(-1j*a*p[0])))
manual_calculation_diagonal = Transpose(np.matmul(prefactor_diagonal, Transpose(psi)))
manual_calculation_plus = np.matmul(U,Transpose(np.matmul(prefactor_plus, Transpose(psi))))
manual_calculation_minus = np.matmul(ConjugateTranspose2(U),Transpose(np.matmul(prefactor_minus, Transpose(psi))))
manual_calculation = manual_calculation_diagonal + manual_calculation_plus + manual_calculation_minus

psi_new2 = applyD_flatten_flatten(Nt, Nx ,gauge_links_t, gauge_links_x,m,Pauli,psi_flatten,
                       down_t_gauge, down_x_gauge, up_t_psi, down_t_psi,
                       down_x_psi, transposed_indices, matmul_indices, columns_matrix2,
                      gauge_links_indices, gamma_indices)


psi_new2 = np.reshape(psi_new2, (Nt,Nx,Nc,Ns))
print(np.sqrt(np.sum((psi_new2-manual_calculation)**2)))

4.965068306494546e-16j


In [53]:
psi_new2-manual_calculation

array([[[[0.+1.11022302e-16j, 0.+5.55111512e-17j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j]],

        [[0.+1.11022302e-16j, 0.+5.55111512e-17j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j]],

        [[0.+1.11022302e-16j, 0.+5.55111512e-17j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j]],

        [[0.+1.11022302e-16j, 0.+5.55111512e-17j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j]]],


       [[[0.+1.11022302e-16j, 0.+5.55111512e-17j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j]],

        [[0.+1.11022302e-16j, 0.+5.55111512e-17j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j],
         [0.+0.00000000e+00j, 0.+0.00000000e+00j]],

        [[0.+1.11022302e-16j, 0.+5.55111512e-17j],
         [0.+0.00

In [54]:
def ConjugateGradient(matrix, eta, targetResidue, iterMax, out_vector):
    r = eta
    p = r
    
    x = np.zeros_like(eta)
    
    counter = 0
    
    resSq = np.dot(r, r)
    
    while (resSq > targetResidue and counter < iterMax):
        # is this gonna be smart enough to do the matrix products inside each spinor vector?
        tmp = SparseMatrixVectorMultiplication(dirac.row_index, dirac.col_index, dirac.values, p)
        ai = resSq / np.dot(tmp, tmp)
        
        # using gamma-5 hermiticity to avoid having to compute or store D^\dagger
        for pos in range(lattice_volume):
            for s in range(Ns):
                tmp[pos][s] = gamma_5 * tmp[pos][s]
        ap  = SparseMatrixVectorMultiplication(dirac.row_index, dirac.col_index, dirac.values, tmp)
        for pos in range(lattice_volume):
            for s in range(Ns):
                ap[pos][s] = gamma_5 * ap[pos][s]
        
        #update residue
        r = r - ai * ap
        
        resSq_old = resSq
        resSq = np.dot(r, r)
        
        bi = resSq / resSq_old
        
        #update solution
        x = x + ai * p
        #update search direction
        p = bi * p + r
        
        counter += 1
    
    out_vector = x
    return counter

# Apply $\gamma_5$ 

In [55]:
def MatMul_gamma5_indices(Nt,Nx,Nc,Ns):
    gamma5_indices = np.repeat(np.repeat(np.arange(Ns)*Ns,Nc).reshape(1,Nc*Ns), Nt*Nx,0).flatten()
    
    size_matrix = Nc*Ns
    indices2 = np.repeat(np.arange(Nc).reshape(1,Nc),Ns,0).flatten()
    indices_lattice = np.arange(Nt*Nx)*size_matrix
    psi_indices = (indices_lattice[:,None] +indices2[None, :]).flatten()


    return gamma5_indices, psi_indices

def ApplyGamma5(gamma5, psi, gamma5_indices, psi_indices, Nc, Ns):
    result = np.zeros(len(gamma5_indices), dtype = 'complex_')
    for column in range(Ns):
        result = result + (gamma5[gamma5_indices +column]*psi[psi_indices + column* Nc])           
    return result

gamma5_indices, psi_indices = MatMul_gamma5_indices(Nt,Nx,Nc,Ns)
gamma5_flatten = gamma5.flatten()
psi = np.random.randn(Nt*Nx,Nc,Ns)

psi_flatten = Transpose(psi).flatten() #BE CAREFUL WITH FLATTENING, FIRST THE RIGHT ORDER OF INDICES

result_flattened = ApplyGamma5(gamma5_flatten, psi_flatten, gamma5_indices, psi_indices,Nc, Ns )
result_reshaped = np.reshape(result_flattened, (Nt*Nx, Ns,Nc))
result = np.matmul(gamma5, Transpose(psi))
np.sqrt(np.sum((result-result_reshaped)**2))

0j

In [56]:
def ConjugateGradient(matrix, eta, targetResidue, iterMax, out_vector):
    r = eta
    p = r
    
    x = np.zeros_like(eta)
    
    counter = 0
    
    resSq = np.dot(r, r)
    
    while (resSq > targetResidue and counter < iterMax):
        # is this gonna be smart enough to do the matrix products inside each spinor vector?
        tmp = SparseMatrixVectorMultiplication(dirac.row_index, dirac.col_index, dirac.values, p)
        ai = resSq / np.dot(tmp, tmp)
        
        # using gamma-5 hermiticity to avoid having to compute or store D^\dagger
        for pos in range(lattice_volume):
            for s in range(Ns):
                tmp[pos][s] = gamma_5 * tmp[pos][s]
        ap  = SparseMatrixVectorMultiplication(dirac.row_index, dirac.col_index, dirac.values, tmp)
        for pos in range(lattice_volume):
            for s in range(Ns):
                ap[pos][s] = gamma_5 * ap[pos][s]
        
        #update residue
        r = r - ai * ap
        
        resSq_old = resSq
        resSq = np.dot(r, r)
        
        bi = resSq / resSq_old
        
        #update solution
        x = x + ai * p
        #update search direction
        p = bi * p + r
        
        counter += 1
    
    out_vector = x
    return counter

In [57]:
def Norm(vector):
    return np.dot(np.conjugate(vector),vector)

In [58]:
Nt = 4
Nx = 4
Nc = 3
Ns = 2
down_t_gauge = roll_t_indices(Nt,Nx,Nc*Nc,1)
down_x_gauge = roll_x_indices(Nt,Nx,Nc*Nc,1)
            
up_t_psi     = roll_t_indices(Nt,Nx,Nc*Ns,-1)
up_x_psi     = roll_x_indices(Nt,Nx,Nc*Ns,-1)
down_t_psi   = roll_t_indices(Nt,Nx,Nc*Ns,1)
down_x_psi   = roll_x_indices(Nt,Nx,Nc*Ns,1)

transposed_indices = Transposed_indices(Nt,Nx,Nc)

matmul_indices, columns_matrix2 = MatMul_indices(Nt,Nx,Nc*Ns)

gauge_links_indices, gamma_indices = Kron_indices(Nt,Nx, Nc,Ns)

lattice_volume = Nt*Nx

U = unitary_group.rvs(3)
gauge_links = np.zeros((lattice_volume, nDim, Nc,Nc),dtype = 'complex_' )
for i in range(lattice_volume):
    for j in range(nDim):
        gauge_links[i,j] = U
        
gamma_t_plus = np.identity(Ns) - Pauli[2] 
gamma_t_minus = np.identity(Ns) + Pauli[2] 
gamma_x_plus = np.identity(Ns) - Pauli[0] 
gamma_x_minus = np.identity(Ns) + Pauli[0]

gamma5_indices, psi_indices = MatMul_gamma5_indices(Nt,Nx,Nc,Ns)

In [59]:
r = np.random.randn(Nt,Nx,Nc,Ns).flatten()
p = r


tmp = ApplyGamma5(gamma5_flatten, psi_flatten, gamma5_indices, psi_indices,Nc, Ns ) #Multiply with psi or p??

tmp = applyD_flatten_flatten(Nt, Nx ,gauge_links_t, gauge_links_x,m,Pauli, tmp,
                               down_t_gauge, down_x_gauge, up_t_psi, down_t_psi,
                               down_x_psi, transposed_indices, matmul_indices, columns_matrix2,
                              gauge_links_indices, gamma_indices)
tmp = ApplyGamma5(gamma5_flatten, tmp, gamma5_indices, psi_indices,Nc, Ns )

alpha_denominator = Norm(tmp)
alpha_nominator = Norm(r)
alpha = alpha_nominator/alpha_denominator

psi_new = psi_flatten + alpha* p
r_new = r - alpha*tmp

beta_nominator = Norm(r_new)
beta = beta_nominator/alpha_nominator
p_new = r_new + beta*p